In [ ]:
import nest_asyncio
nest_asyncio.apply()  # Fix for running async code in Jupyter

from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient, load_mcp_tools
import os
import asyncio
import requests
from pprint import pprint

# Function to get JWT token
def get_jwt_token(username="admin", password="password"):
    try:
        response = requests.post(
            "http://localhost:8000/token",  # Update port if needed
            params={"username": username, "password": password}
        )
        
        if response.status_code == 200:
            token = response.json()["access_token"]
            print(f"Successfully obtained token: {token[:20]}...")
            return token
        else:
            print(f"Error getting token: {response.status_code}")
            print(response.text)
            return None
    except Exception as e:
        print(f"Exception getting token: {str(e)}")
        return None

# Get a fresh token
token = get_jwt_token()
if not token:
    print("Failed to get JWT token. Exiting.")
else:
    # Test the token with a direct request
    try:
        test_response = requests.get(
            "http://localhost:8000/teas",
            headers={"Authorization": f"Bearer {token}"}
        )
        print(f"Test request status: {test_response.status_code}")
        if test_response.status_code == 200:
            print("Token authentication successful!")
        else:
            print(f"Token authentication failed: {test_response.text}")
    except Exception as e:
        print(f"Test request error: {str(e)}")

    llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

    # Main function to run the agent with JWT authentication
    async def main():
        client = MultiServerMCPClient({
            "tea_api": {
                "url": "http://localhost:8080/sse",
                "transport": "sse",
                "headers": {
                    "Authorization": f"Bearer {token}"
                }
            }
        })

        async with client.session("tea_api") as session:
            # Load tools
            tools = await load_mcp_tools(session)
            pprint([tool.name for tool in tools])
            
            # Create a system prompt that explains authentication
            system_prompt = """You are a tea assistant that helps users manage their tea inventory.
            
            You have access to a tea shop API with the following capabilities:
            - View all teas in the shop
            - Add new teas to the collection
            - Update existing teas
            - Delete teas from the collection
            
            Authentication is already handled for you. Help the user manage their tea collection.
            """
            
            # Create the agent with the authentication-aware prompt
            agent = create_react_agent(llm, tools=tools, prompt=system_prompt)
            
            # Define the user's request
            create_message = "Please add 3 more new items in the list and mention its origin as India" 
            
            # Set up the initial state with the user's message
            state = {"messages": [HumanMessage(content=create_message)]}
            
            # Invoke the agent
            print("\nProcessing request...")
            result = await agent.ainvoke(state)
            print("\n💬 Agent Response:\n", result)

    # Run the async function
    await main()


Successfully obtained token: eyJhbGciOiJIUzI1NiIs...
Test request status: 200
Token authentication successful!
['login_for_access_token_token_post',
 'root',
 'get_teas_teas_get',
 'create_tea_teas_post',
 'update_tea_teas',
 'delete_tea_teas']

Processing request...

💬 Agent Response:
 {'messages': [HumanMessage(content='Please add 3 more new items in the list and mention its origin as India', additional_kwargs={}, response_metadata={}, id='9f2eba1c-bd5b-4ef9-b3ac-2e25ff66572f'), AIMessage(content='Could you please provide the names of the three new teas you would like to add to the collection?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 911, 'total_tokens': 932, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_finger